In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
# Langchain tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANCHAIN_PROJECT_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT") 

In [4]:
## data ingestion from the website we need to scrape the data 
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/administration/concepts")
loader

In [6]:
docs =  loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content="\n\n\n\n\nConcepts | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationConceptual GuideOn this pageConcepts\nThis conceptual guide covers topics related to managing users, organizations, and workspaces within L

In [7]:
# Load Data --> Docs ---> Divide our text into chunks --> vector --> Vector Embedding -- > Vector Store 
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter =  RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap= 200)
documents = text_splitter.split_documents(docs)

In [8]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content='Concepts | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scal

In [22]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings = GoogleGenerativeAIEmbeddings( # This will give us 768 dim so we can go for ollama 
#     model="models/embedding-001"   
# )
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model = "gemma:2b"
)


C:\Users\Nirav\AppData\Local\Temp\ipykernel_21376\581666082.py:8: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


In [23]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents,embeddings)

In [24]:
len(vectorstoredb.index_to_docstore_id)


33

In [25]:
print(len(embeddings.embed_query("test")))


2048


In [26]:
print(len(vectorstoredb.index_to_docstore_id))


33


In [27]:
vectors = vectorstoredb.index.reconstruct_n(0, vectorstoredb.index.ntotal)
print(vectors)

[[ 0.7070765  -0.04676775 -1.1672823  ...  0.32437387 -1.233724
   1.3048584 ]
 [ 0.40202475 -0.561891   -0.9718819  ... -3.0186098  -1.5597785
  -0.12363089]
 [-0.6352203  -1.4135911  -0.785432   ... -4.0504475  -0.19131994
   0.63039935]
 ...
 [-0.7365145   0.09503919 -0.8498155  ... -0.17175633  1.1147379
  -0.3621799 ]
 [-0.7230354  -1.0952692   0.7877405  ... -2.1404414  -0.60940826
   0.03427316]
 [ 0.33645463 -0.35699156  0.3056506  ... -1.3394234  -1.5823153
   0.48569438]]


#### **To see the vector of the given content**

In [28]:
for i in range(vectorstoredb.index.ntotal):
    vector = vectorstoredb.index.reconstruct(i)
    doc_id = vectorstoredb.index_to_docstore_id[i]
    doc_content = vectorstoredb.docstore._dict[doc_id].page_content
    print(f"Doc: {doc_content}\nVector: {vector[:5]}...")  # print first 5 dims


Doc: Concepts | 🦜️🛠️ LangSmith
Vector: [ 0.7070765  -0.04676775 -1.1672823   1.6084164   0.23011196]...
Doc: Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationConceptual GuideOn this pageConcepts
This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.
Resource Hierarchy​
Organizations​
An organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company. An organization can have multiple worksp

In [33]:
query = "Organization roles are distinct from the Enterprise feature (RBAC) "
result = vectorstoredb.similarity_search(query)
result[0].page_content

'Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces. An Organization Admin has Admin access to all workspaces in an organization\nOrganization User may read organization information but cannot execute any write actions at the organization level. An Organization User can be added to a subset of workspaces and assigned workspace roles as usual (if RBAC is enabled), which specify permissions at the workspace level.'

In [34]:
# Retrieval Chain , Document Chain  {Most important}
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>
"""
)

document_chain =  create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001A8F8788E10>, default_metadata=(), model_kwargs={})
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [37]:
from langchain_core.documents import Document

document_chain.invoke(
    {
        "input": "Organization roles are distinct from the Enterprise feature (RBAC)",
        "context": [
            Document(
                page_content="Organization roles are distinct from the Enterprise feature (RBAC), Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces"
            )
        ]
    }
)

'An Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces.'

However we want the documents to first come the retriever we just set up. That way , we can use the retriever to dynamically select the most relevant documents and pass those in for a given question

In [ ]:
## Input --> Retriever 
vectorstoredb

In [41]:
# Create a retriever 
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain =  create_retrieval_chain(retriever, document_chain) 

In [42]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A89DD5F110>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | Chat

In [43]:
# Get the response from the LLM
retrieval_chain.invoke({"input":"Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces"})


{'input': 'Organization Admin grants full access to manage all organization configuration, users, billing, and workspaces',
 'context': [Document(id='343b9e69-3f0d-4feb-a013-be788b8c5bc3', metadata={'source': 'https://docs.smith.langchain.com/administration/concepts', 'title': 'Concepts | 🦜️🛠️ LangSmith', 'description': 'This conceptual guide covers topics related to managing users, organizations, and workspaces within LangSmith.', 'language': 'en'}, page_content='All traces limit\nExtended data retention traces limit\n\nThese let you limit the number of total traces, and extended data retention traces respectively.\nProperties of usage limiting\u200b\nUsage limiting is approximate, meaning that we do not guarantee the exactness of the limit. In rare cases, there\nmay be a small period of time where additional traces are processed above the limit threshold before usage limiting\nbegins to apply.\nSide effects of extended data retention traces limit\u200b\nThe extended data retention tr